<a href="https://colab.research.google.com/github/ChauLinHuang/DA_Term_Project_Smoke_Detection_through_Satellite_Imagery/blob/main/DA_Term_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary modules.

In [9]:
from google.colab import drive

from os import path, listdir
from os.path import join, isdir, getsize
from pathlib import Path
from random import shuffle, seed, randrange
import pandas as pd
from sklearn.model_selection import train_test_split

import sys
import math
import re
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json

# This module is manually in this script
# import mrcnn
import xml.etree.ElementTree as ET

%matplotlib inline 

# Import datasets

In [12]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [13]:
dataset_dir = '/content/gdrive/My Drive/RPI_ITWS/Fall 2020/Data Analytics/Project/datasets/smoke_labeled/'

# Image data exploration




In [ ]:
print('# File sizes')
for imageFile in listdir(dataset_dir):
  imageDir = join(dataset_dir, imageFile)
  imageSize = getsize(imageDir) / 1000000
  print(imageFile + '\t\t' + str(round(imageSize, 2)) + 'MB' )
  

In [5]:
allImgSizes = []
for imageFile in listdir(dataset_dir):
  imageDir = join(dataset_dir, imageFile)
  imageSize = getsize(imageDir) / 1000000
  allImgSizes.append(imageSize)
    
# Total memories
print('Total image memories: ', round(sum(allImgSizes), 3), ' MB')

# Max
print('Image with max memory with : ', round(max(allImgSizes), 3), ' MB')

# Min
print('Image with min memory with : ', min(allImgSizes), ' MB')


Total image memories:  228.376  MB
Image with max memory with :  19.245  MB
Image with min memory with :  0.000266  MB


# Methods to bind pairs of raw and tagged images

In [6]:
class ImagePair:
  def __init__(self, dataset_dir: str):
    if not path.isdir(dataset_dir):
      raise NameError('ImagePair.__init__ exception: {} does not exist'.format(dataset_dir))

    self._all_images = listdir(dataset_dir)
    self._numImgs = len(self._all_images)
    self._imgPairList = []
  
  def pair(self):
    bmp_files = set()
    tif_files = set()
    for imageFile in self._all_images:
      baseName = Path(imageFile).stem
      if imageFile.endswith('.bmp'):
        bmp_files.add(baseName)
      elif imageFile.endswith('.tif'):
        tif_files.add(baseName)
      else:
        pass

    matched = bmp_files.intersection(tif_files)

    for baseName in matched:
      bmp_f = baseName + '.bmp'
      tif_f = baseName + '.tif'
      json_f = baseName + '.json'
      self._imgPairList.append( (bmp_f, tif_f, json_f) )

  def getPairs(self) -> list():
    return self._imgPairList



# Train test split

Let's use a 80-20 split.

In [7]:
def split_to_train_test_imgs (dataset_dir, train_size = None, test_size = None):
  '''
  @brief: Manually split the single dataset into a train and a test dataset.
  @param train_size: Should be a float number beteen 0 and 1 which represents the
          the train set size percentage in the original dataset
  @param test_size: Should be a float number between 0 and 1 which stands for the 
          test set size percentage in the original dataset
  @return train_set: first component in the tuple
  @return test_set: second component in the tuple
  '''
  if train_size is None and test_size is None:
    train_size, test_size = 0.5, 0.5
  elif train_size is None and test_size is not None:
    train_size = 1 - test_size
  elif test_size is None and train_size is not None:
    test_size = 1 - train_size
  
  if train_size + test_size != 1:
    raise ValueError('train_size and test_size do not come in reasonable ratio. Please make sure they agree with each other.')

  matcher = ImagePair(dataset_dir)
  matcher.pair()
  image_pairs = matcher.getPairs()

  total_num = len( image_pairs )
  train_num = int( train_size * total_num ) # Note: Maybe ceil() could be a better fit?

  return image_pairs[:train_num], image_pairs[train_num:]


In [76]:
# Driver code
train_set, test_set = split_to_train_test_imgs(dataset_dir, train_size=0.8)

print(train_set[0][0])



281
71


# Cross-validation

We are using Stratified K-Fold approach to distribute image pairs.

In [83]:
class KFold_cross_validation:
  '''
  Prepare the splitted datasets using stratified
  k-fold cross validation.

  @return folds: is a 2D list. The rows represent the folds, each fold contains
         the corresponding amount of image pairs.

  TODO: replace Python's list with np.array or pd.DataFrame
  '''
  def __init__(self, imagePairs: list(), num_folds = None, _do_stratify = None):
    self._imagePairs = list(imagePairs)
    self._num_pairs = len( self._imagePairs )
    if num_folds is None:
      self._folds = 10
    else:
      self._folds = num_folds
    if _do_stratify is None:
      self._do_stratify = True
    else:
      self._do_stratify = _do_stratify
    self._splitted = []
  
  def set_folds(self, folds: int) -> None:
    self._folds = folds
  
  def set_stratify(self, do_stratify: bool) -> None:
    self._do_stratify = do_stratify

  def shuffle(self, imagePairs = None) -> None:
    if imagePairs is None:
      shuffle(self._imagePairs)
    else:
      shuffle(imagePairs)

  def split(self, folds = None, do_stratify = None):
    if folds is None:
      folds = self._folds
    if do_stratify is None:
      do_stratify = self._do_stratify

    fold_size = int( self._num_pairs / folds )

    for i in range( folds ):
      foldChunk = []
      while len(foldChunk) < fold_size:
        if do_stratify:
          index = randrange( len(self._imagePairs) ) # use the current, smaller size of the list
          foldChunk.append( self._imagePairs.pop(index) )

        else:
          foldChunk.append( self._imagePairs.pop() )
      self._splitted.append(foldChunk)
    
    return self._splitted


We will do cross validation across 5 folds of data.

In [86]:
# driver code
train_skf = KFold_cross_validation(train_set, num_folds = 5)
train_skf.shuffle()
train_folds = train_skf.split()

test_skf = KFold_cross_validation(test_set, num_folds = 5)
test_skf.shuffle()
test_folds = test_skf.split()

print('There are {} train folds each with {} image pairs.'.format( len(train_folds), len(train_folds[0]) ))
print('There are {} test folds each with {} image pairs.'.format( len(test_folds), len(test_folds[0]) ))

There are 5 train folds each with 56 image pairs.
There are 5 test folds each with 14 image pairs.


Now we have the trin_X, test_X, train_y, and test_y for all 5 folds.

# Mask R-CNN

We will train and use Mask R-CNN to segment out the smoke in the images. 
The following bash script snippet does not need to be run if this is not the your first time running this Jupyter notebook.

In [26]:
!pip install opencv-contrib-python

We are using the Mask-RCNN repo compatible with TF 2.0. Eventhough most tutorials exemplify the package's application with the TF1.0-compatible counterpart, the codes for both version of Mask R-CNN are mostly the same.

In [ ]:
%%bash
# NOTE: Don't run the script below if you've already done it.
# Install the ahmedfgad's version of Mask R-CNN
git clone https://github.com/ahmedfgad/Mask-RCNN-TF2.git
cd /content/Mask-RCNN-TF2
pip3 install -r requirements.txt
python setup.py install

Test if Mask R-CNN is properly installed

In [17]:
# This is a dirty way of importing custom modules
Mask_RCNN_repo_dir = '/content/Mask-RCNN-TF2'
sys.path.append(Mask_RCNN_repo_dir)


print('The Mask R-CNN modules can be imported')

The Mask R-CNN modules can be imported


In [18]:
import mrcnn

# Import Mask RCNN
from mrcnn.config import Config
from mrcnn import utils, visualize
# import mrcnn.model as modellib, log
from mrcnn.model import log
import mrcnn.model as modellib

In [19]:
# Root directory of the project
ROOT_DIR = path.abspath('/content/')

# Directory to save logs and trained model's parameters
MODEL_ARTIFACT_DIR = join(ROOT_DIR, "mask_rcnn_logs")

# Local path to trained weights file
SMOKE_MODEL_PATH = join(ROOT_DIR, "mask_rcnn_smoke.h5")
COCO_MODEL_PATH = join(ROOT_DIR, 'mask_rcnn_coco.h5')

# Download COCO trained weights from Release if needed
if not path.exists(COCO_MODEL_PATH):
  utils.download_trained_weights(COCO_MODEL_PATH)

... done downloading pretrained model!


# Create adequate configurations for the model

In [20]:
class SmokeDetectionConfig (Config):
	# give the configuration a recognizable name
	NAME = "smoke_detection"
	# set the number of GPUs to use along with the number of images
	# per GPU
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1
	# number of classes (we would normally add +1 for the background
	# but the background class is *already* included in the class
	# names)
	NUM_CLASSES = 1 + 1

In [21]:
# initialize the inference configuration
config = SmokeDetectionConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

# Load in Dataset

Create the dataset loader and override three methods:
- load_dataset()
- load_mask()
- extract_mask()
- image_reference()

In [22]:
class SmokeDataset (utils.Dataset):
  def load_dataset(self, dataset_dir):
    self.add_class('dataset', 1, 'smoke')

    # find all raw images
    _raw_format = '.tif'
    for i, filename in enumerate( listdir(dataset_dir) ):
      if filename.endswith(_raw_format):
        self.add_image('dataset',
                       image_id=i,
                       path=join(dataset_dir, filename),
                       annotation=join(dataset_dir, filename.replace(_raw_format, '.xml')))
    
  # extract bounding boxes
  # TODO
  def extract_boxes(self, filename):
    '''
    Since I don't have the bounding box labels, I don't have any tagged images
    for this category. Gotta leave it blank for the moment. Will add this feature
    back later.
    '''
    pass

  # Extract custom instance segmentations
  def extract_mask(self, filename):
    json_file = join(filename)
    with open(json_file) as f:
        img_anns = json.load(f)
        
    masks = np.zeros([600, 800, len(img_anns['shapes'])], dtype='uint8')
    classes = []
    for i, anno in enumerate(img_anns['shapes']):
        mask = np.zeros([600, 800], dtype=np.uint8)
        cv2.fillPoly(mask, np.array([anno['points']], dtype=np.int32), 1)
        masks[:, :, i] = mask
        classes.append(self.class_names.index(anno['label']))
    return masks, classes


  # Load the masks for an image
  def load_mask(self, image_id):
    # get details of image
    info = self.image_info[image_id]
    # define box file location
    path = info['annotation']
    # load XML
    boxes, classes, w, h = self.extract_boxes(path)
    # create one array for all masks, each on a different channel
    # WIP: Maybe we only need one channel, since its black and white
    num_boxes = len(boxes)
    masks = np.zeros([h, w, num_boxes], dtype = 'uint8')

    # create masks
    for i in range(num_boxes):
      box = boxes[i]
      row_s, row_e = box[1], box[3]
      col_s, col_e = box[0], box[2]
      masks[row_s : row_e, col_s : col_e, i] = 1

    return masks, np.asarray(classes, dtype='int32')
  
  def image_reference (self, image_id):
    info = self.image_info[image_id]
    return info['path']

### Significant variables:
train_dataset \
test_dataset

In [26]:
# Create training and validation dataset
# train set
train_dataset_dir = '/content/gdrive/MyDrive/RPI_ITWS/Fall 2020/Data Analytics/Project/datasets/train'
train_dataset = SmokeDataset()
train_dataset.load_dataset(train_dataset_dir)
train_dataset.prepare()

print('Train dataset: ', len(train_dataset.image_ids), ' samples.')

# test or validation set
test_dataset_dir = '/content/gdrive/MyDrive/RPI_ITWS/Fall 2020/Data Analytics/Project/datasets/test'
val_dataset = SmokeDataset()
val_dataset.load_dataset(test_dataset_dir)
val_dataset.prepare()

print('Test dataset: ', len(val_dataset.image_ids), ' samples.')

Train dataset:  311  samples.
Test dataset:  71  samples.


In [27]:
# Load and display random samplers
num_features = 1
image_ids = np.random.choice(train_dataset.image_ids, num_features)

for image_id in image_ids:
  image = train_dataset.load_image(image_id)
  mask, class_ids = train_dataset.load_mask(image_id)
  visualize.display_top_masks(image, mask, class_ids, dataset_train_class_names)
  

TypeError: ignored

# Create model

### Significant variables
model \
init_with

In [40]:
# Create model in training mode
model = modellib.MaskRCNN(mode='training',
                          config=config,
                          model_dir=MODEL_ARTIFACT_DIR)

In [ ]:
# which weight to start with?
init_with = 'coco' # imagenet, coco, or last

if init_with == 'imagenet':
  model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == 'coco':
  '''Load weights trained on MS COCO, but skip layers that
    are different due to the different number of classes'''
  model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
  # Load the last model you trained and continue training
  model.load_weights(model.find_last(), by_name=True)

# Training

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(train_dataset, val_dataset, 
            learning_rate=config.LEARNING_RATE, 
            epochs=5, 
            layers='heads')

In [ ]:
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")

model_path = model.find_last()

# Load trained weights
print('Loading weights from ', model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=10, 
            layers="all")

# Detection

In [ ]:
class InferenceConfig(SmokeDetectionConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_ARTIFACT_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

# Visualization

In [33]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Test on a random image
image_id = random.choice(val_dataset.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(val_dataset, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

# Evaluation

Metrics: Average Precision (AP)

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = val_dataset.image_ids
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))

# References

[1] Initial EDA - Image Processing | Kaggle \
https://www.kaggle.com/cc786537662/initial-eda-image-processing

[2] TannerGilbert/MaskRCNN-Object-Detection-and-Segmentation: Train your own custom MaskRCNN Object Detection and Instance Segmentation model. \
https://github.com/TannerGilbert/MaskRCNN-Object-Detection-and-Segmentation